<a href="https://colab.research.google.com/github/DavidP0011/apps_functions/blob/main/app_pdf_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INICIALIZACIÓN

In [ ]:
def install_dpm_repos(config: dict) -> None:
    """
    Instala repositorios DPM y lista las funciones de cada paquete instalado.

    El diccionario de configuración debe tener la clave:
        - github_repo_url_list: Lista de URLs de repositorios GitHub
          Ejemplo:
            {
                "github_repo_url_list": [
                  "https://github.com/DavidP0011/common_functions",
                  "https://github.com/DavidP0011/utils_functions",
                  "https://github.com/DavidP0011/etl_functions"
                  ]
            }

    La función procede en dos pasos para cada repositorio:
      1. Se instala de manera “normal” (pip install --upgrade --no-cache-dir git+<repo_url>)
         para instalar las dependencias si faltan.
      2. Se fuerza la reinstalación de los archivos .py con --force-reinstall y --no-deps
         (pip install --upgrade --force-reinstall --no-deps --no-cache-dir git+<repo_url>).

    Luego, se detecta dinámicamente el nombre del paquete instalado y se recorre
    cada paquete para imprimir las funciones definidas.
    """
    import subprocess
    import sys
    import importlib
    import pkgutil
    import inspect

    # Para detectar distribuciones y top-level packages
    try:
        import importlib.metadata as metadata
    except ImportError:
        # Si la versión de Python es muy antigua:
        import importlib_metadata as metadata

    def _validate_config(cfg: dict) -> list:
        """Valida que la configuración tenga la clave 'github_repo_url_list' con una lista."""
        if "github_repo_url_list" not in cfg:
            raise ValueError("[VALIDATION [ERROR ❌]] La configuración debe contener la clave 'github_repo_url_list'.")
        if not isinstance(cfg["github_repo_url_list"], list):
            raise ValueError("[VALIDATION [ERROR ❌]] La clave 'github_repo_url_list' debe ser una lista de URLs.")
        return cfg["github_repo_url_list"]

    def _install_repo(repo_url: str) -> None:
        """
        Instala un repositorio en dos pasos:
          1. Instala el repositorio de forma normal (para que se instalen las dependencias si aún no están).
          2. Fuerza la reinstalación del código (los archivos .py) sin tocar las dependencias (--no-deps).
        """
        print(f"\n[START ▶️] Instalación inicial (con dependencias) del repositorio: {repo_url}", flush=True)
        cmd_normal = [
            sys.executable, "-m", "pip", "install", "-q",
            "--upgrade", "--no-cache-dir",
            f"git+{repo_url}"
        ]
        try:
            subprocess.check_call(cmd_normal)
            print(f"[SUCCESS ✅] Instalación inicial completada.", flush=True)
        except subprocess.CalledProcessError as e:
            print(f"[ERROR ❌] Falló la instalación inicial del repositorio. Detalle: {e}", flush=True)

        print(f"\n[START ▶️] Forzando reinstalación del código del repositorio: {repo_url}", flush=True)
        cmd_force = [
            sys.executable, "-m", "pip", "install", "-q",
            "--upgrade", "--force-reinstall", "--no-deps", "--no-cache-dir",
            f"git+{repo_url}"
        ]
        try:
            subprocess.check_call(cmd_force)
            print(f"[SUCCESS ✅] Reinstalación del código completada.", flush=True)
        except subprocess.CalledProcessError as e:
            print(f"[ERROR ❌] Falló la reinstalación del código. Detalle: {e}", flush=True)

    def _detect_installed_package_name(before_set: set, repo_url: str) -> str:
        """
        Compara las distribuciones antes y después de la instalación:
        - before_set: conjunto de nombres de distribución (metadata["Name"]) previos.
        - repo_url: URL del repositorio (para intentar emparejar por substring).
        Devuelve el nombre de paquete (top-level) para importar.
        """
        # Obtenemos el conjunto actual de distribuciones instaladas
        after_set = {dist.metadata["Name"] for dist in metadata.distributions()}

        # Identificamos cuáles son las nuevas distribuciones
        nuevas = list(after_set - before_set)

        repo_name = repo_url.rstrip("/").split("/")[-1].lower()

        if not nuevas:
            # Si no hay distribuciones nuevas, intentamos emparejar alguna existente por substring
            candidatos = [dist.metadata["Name"] for dist in metadata.distributions() if repo_name in dist.metadata["Name"].lower()]
            if candidatos:
                elegida_dist = candidatos[0]
                print(f"[INFO ℹ️] No se detectó distribución nueva. Se asume '{elegida_dist}' para '{repo_name}'.", flush=True)
            else:
                print(f"[WARN ⚠️] No se detectó nueva distribución ni coincidencias para '{repo_name}'. Se usará '{repo_name}' por defecto.", flush=True)
                return repo_name
        else:
            # Si hay más de una nueva, intentamos filtrar por substring
            if len(nuevas) > 1:
                filtradas = [d for d in nuevas if repo_name in d.lower()]
                elegida_dist = filtradas[0] if filtradas else nuevas[0]
                print(f"[INFO ℹ️] Se detectaron múltiples distribuciones nuevas {nuevas}. Se elige '{elegida_dist}'.", flush=True)
            else:
                elegida_dist = nuevas[0]
            print(f"[INFO ℹ️] Se detectó la nueva distribución '{elegida_dist}'.", flush=True)

        # Ahora obtenemos el top-level package desde la metadata
        try:
            dist_info = metadata.distribution(elegida_dist)
            top_level_txt = dist_info.read_text("top_level.txt")
            if top_level_txt:
                top_levels = [line.strip() for line in top_level_txt.splitlines() if line.strip()]
                paquete = top_levels[0] if top_levels else elegida_dist
                print(f"[INFO ℹ️] Para la distribución '{elegida_dist}', se detectó el paquete top-level: '{paquete}'", flush=True)
                return paquete
            else:
                print(f"[WARN ⚠️] 'top_level.txt' vacío para '{elegida_dist}'. Se usará '{elegida_dist}' como paquete.", flush=True)
                return elegida_dist
        except Exception as e:
            # Si no existe top_level.txt o falla, devolvemos el nombre de la distribución
            print(f"[ERROR ❌] No se pudo leer 'top_level.txt' de '{elegida_dist}': {e}. Usando '{elegida_dist}'.", flush=True)
            return elegida_dist

    def _list_functions(package) -> None:
        """
        Recorre todos los módulos y submódulos del paquete e imprime las funciones definidas en cada uno.
        """
        print(f"\n[START ▶️] Funciones en el paquete: {package.__name__}", flush=True)
        for finder, module_name, is_pkg in pkgutil.walk_packages(package.__path__, package.__name__ + "."):
            try:
                modulo = importlib.import_module(module_name)
                funciones = [
                    nombre for nombre, objeto in inspect.getmembers(modulo, inspect.isfunction)
                    if inspect.getmodule(objeto) == modulo
                ]
                print(f"\n Módulo: {module_name}", flush=True)
                if funciones:
                    for funcion in funciones:
                        print("  -", funcion, flush=True)
                else:
                    print("  (No se encontraron funciones definidas)", flush=True)
            except Exception as e:
                print(f"[ERROR ❌] Falló al importar el módulo {module_name}: {e}", flush=True)

    # Mensaje de inicio del proceso global
    print("🔹🔹🔹 Instalación de Repositorios DPM y listado de funciones 🔹🔹🔹", flush=True)

    # Validación de la configuración
    github_repo_url_list = _validate_config(config)

    for repo_url in github_repo_url_list:
        # Tomamos snapshot de distribuciones antes de instalar
        antes = {dist.metadata["Name"] for dist in metadata.distributions()}

        # Instalamos el repositorio en dos pasos
        _install_repo(repo_url)

        # Detectamos automáticamente el nombre del paquete instalado
        package_name = _detect_installed_package_name(antes, repo_url)

        # Importamos el paquete y listamos sus funciones
        try:
            print(f"\n\n🔹🔹🔹 Importando el paquete: {package_name} 🔹🔹🔹", flush=True)
            if package_name in sys.modules:
                del sys.modules[package_name]
            importlib.invalidate_caches()
            paquete = importlib.import_module(package_name)
            _list_functions(paquete)
        except Exception as e:
            print(f"❌ [ERROR] Falló al importar el paquete '{package_name}': {e}", flush=True)

    print("\n\n🔹🔹🔹 [FINISHED ✅] Proceso completado. 🔹🔹🔹", flush=True)


In [ ]:
# @title GIBHUB INSTALL DPM FUNCTIONS
config = {
    "github_repo_url_list": [
        # "https://github.com/DavidP0011/common_functions",
        "https://github.com/DavidP0011/utils_functions",
        # "https://github.com/DavidP0011/etl_functions",
        "https://github.com/DavidP0011/apps_functions"
    ]
}


install_dpm_repos(config)

🔹🔹🔹 Instalación de Repositorios DPM y listado de funciones 🔹🔹🔹

[START ▶️] Instalación inicial (con dependencias) del repositorio: https://github.com/DavidP0011/common_functions
[SUCCESS ✅] Instalación inicial completada.

[START ▶️] Forzando reinstalación del código del repositorio: https://github.com/DavidP0011/common_functions
[SUCCESS ✅] Reinstalación del código completada.
[INFO ℹ️] No se detectó distribución nueva. Se asume 'dpm_common_functions' para 'common_functions'.
[INFO ℹ️] Para la distribución 'dpm_common_functions', se detectó el paquete top-level: 'dpm_common_functions'


🔹🔹🔹 Importando el paquete: dpm_common_functions 🔹🔹🔹

[START ▶️] Funciones en el paquete: dpm_common_functions

 Módulo: dpm_common_functions.dpm_GCP_ini_utils
  - _ini_authenticate_API
  - ini_environment_identification
  - ini_google_drive_instalation

[START ▶️] Instalación inicial (con dependencias) del repositorio: https://github.com/DavidP0011/utils_functions
[SUCCESS ✅] Instalación inicial comple

In [ ]:
# @title IDENTIFICACION DE ENTORNO, INSTALACIÓN GOOGLE DRIVE

from dpm_common_functions.dpm_GCP_ini_utils import ini_environment_identification, ini_google_drive_instalation

# Detectar el entorno de ejecución
ini_environment_identificated = ini_environment_identification()
print(f"[INFO ℹ️] Entorno detectado: {ini_environment_identificated}", flush=True)

GCP_json_keyfile_local = r"D:\PYTHON\proyectos\api_keys\animum-dev-apps-venv-python-local.json"
GCP_json_keyfile_colab = "/content/drive/MyDrive/ANIMUM DIRECCION/DIRECCION BI/NOTEBOOKS/api_keys/animum-dev-datawarehouse-google-colab.json"
GCP_json_keyfile_GCP_secret_id = "notebook-vm"

# Montar Google Drive si entorno_identificado_str es Colab
params = {"entorno_identificado_str": ini_environment_identificated}
ini_google_drive_instalation(params)

[INFO ℹ️] Entorno detectado: LOCAL
[INFO ℹ️] El entorno 'LOCAL' no requiere montaje de Google Drive.


In [ ]:
# @title IMPORTACIÓN DE LIBRERÍAS WHISPER Y FFMPEG

# Verificar e instalar el paquete 'whisper'
try:
    import whisper
except ImportError:
    print("El paquete 'whisper' no está instalado. Procediendo con la instalación...")
    !pip install git+https://github.com/openai/whisper.git
    import whisper

# Verificar e instalar el paquete 'ffmpeg-python'
try:
    import ffmpeg
except ImportError:
    print("El paquete 'ffmpeg-python' no está instalado. Procediendo con la instalación...")
    !pip install ffmpeg-python

# Instalar y verificar 'ffmpeg' a nivel del sistema
import os
if os.system("ffmpeg -version") != 0:
    print("FFmpeg no está instalado. Procediendo con la instalación...")
    !apt-get update
    !apt-get install -y ffmpeg
    !ffmpeg -version
else:
    print("FFmpeg ya está instalado.")



FFmpeg ya está instalado.


# EJECUCIONES

In [ ]:
# @title RECOPILACIÓN DE RUTAS DE VIDEO

# Ejemplo de configuración
config = {
    "video_files_root_path": r"\\animum.local\Produccion\LOL-Contenido\ASIGNATURAS",
    "video_files_target_search_folder": ["CAPITULOS EXPORTAR","CAPITULO EXPORTAR"],
    "video_files_target_search_extension": [".mp4"],

    "ini_environment_identificated": ini_environment_identificated,
    "json_keyfile_local": GCP_json_keyfile_local,
    "json_keyfile_colab": GCP_json_keyfile_colab,
    "json_keyfile_GCP_secret_id": GCP_json_keyfile_GCP_secret_id,
}

# Ejecución de la función con manejo de errores interno
from dpm_apps_functions.dpm_apps_utils import files_path_collect_df
df_videos = files_path_collect_df(config)
df_videos


[PROCESS START ▶️] Iniciando la recolección de archivos.
[VALIDATION SUCCESS ✅] Parámetros validados correctamente.
[FILE SEARCH START ▶️] Buscando archivos en '\\animum.local\Produccion\LOL-Contenido\ASIGNATURAS\COLORISTA' con extensiones: ['.mp4'].
[LOCATED FILE INFO ℹ️] Archivo localizado: 010 Como funciona el curso - 010 Como funciona el curso.mp4 (Ruta: \\animum.local\Produccion\LOL-Contenido\ASIGNATURAS\COLORISTA\T000 COMO FUNCIONA EL CURSO\L010 COMO FUNCIONA EL CURSO\CAPITULOS EXPORTAR\010 Como funciona el curso - 010 Como funciona el curso.mp4)
[LOCATED FILE INFO ℹ️] Archivo localizado: 010 Teoria del color - 010 Que es el color.mp4 (Ruta: \\animum.local\Produccion\LOL-Contenido\ASIGNATURAS\COLORISTA\T010 NOCIONES SOBRE EL COLOR\L010 TEORIA DEL COLOR\CAPITULOS EXPORTAR\010 Teoria del color - 010 Que es el color.mp4)
[LOCATED FILE INFO ℹ️] Archivo localizado: 010 Teoria del color - 020 El color tecnico.mp4 (Ruta: \\animum.local\Produccion\LOL-Contenido\ASIGNATURAS\COLORISTA\T01

,file_name,file_path,file_creation_date,file_last_modified_date,file_scrap_date,file_size_mb,duration_hms,duration_ms,video_codec,video_bitrate_kbps,video_fps,video_resolution,audio_codec,audio_bitrate_kbps,audio_channels,audio_sample_rate_hz
0,010 Como funciona el curso - 010 Como funciona...,\\animum.local\Produccion\LOL-Contenido\ASIGNA...,2021-02-09 08:52:29.217577,2021-02-09 08:52:34.048862,2025-06-12 15:35:58.934822,208,00:15:28,928200,h264,1561,25.0,1920x1080,aac,317,2,48000
1,010 Teoria del color - 010 Que es el color.mp4,\\animum.local\Produccion\LOL-Contenido\ASIGNA...,2021-02-23 10:47:38.413409,2021-02-23 10:47:39.390923,2025-06-12 15:35:59.048482,23,00:01:31,91600,h264,2017,25.0,1920x1080,aac,125,1,48000
2,010 Teoria del color - 020 El color tecnico.mp4,\\animum.local\Produccion\LOL-Contenido\ASIGNA...,2021-02-23 10:48:28.215291,2021-02-23 10:48:28.315301,2025-06-12 15:35:59.152619,3,00:01:05,65840,h264,300,25.0,1920x1080,aac,125,1,48000
3,010 Teoria del color - 030 Percepcion del colo...,\\animum.local\Produccion\LOL-Contenido\ASIGNA...,2021-02-23 10:49:46.757849,2021-02-23 10:49:47.756120,2025-06-12 15:35:59.265183,24,00:02:49,169920,h264,1059,25.0,1920x1080,aac,125,1,48000
4,010 Teoria del color - 040 Colores primarios y...,\\animum.local\Produccion\LOL-Contenido\ASIGNA...,2021-02-23 10:52:13.366395,2021-02-23 10:52:14.819046,2025-06-12 15:35:59.385727,62,00:04:30,270720,h264,1793,25.0,1920x1080,aac,125,1,48000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,020 Gestion Color Management - 100 Flujos de t...,\\animum.local\Produccion\LOL-Contenido\ASIGNA...,2024-09-24 11:46:32.062783,2024-09-24 11:46:38.420203,2025-06-12 15:36:24.998454,252,00:16:36,996320,h264,1999,25.0,1920x1080,aac,125,1,44100
208,020 Gestion Color Management - 100 Flujos de t...,\\animum.local\Produccion\LOL-Contenido\ASIGNA...,2024-09-24 11:48:34.229124,2024-09-24 11:48:37.074637,2025-06-12 15:36:25.112627,112,00:07:24,444960,h264,1999,25.0,1920x1080,aac,125,1,44100
209,020 Gestion Color Management - 100 Flujos de t...,\\animum.local\Produccion\LOL-Contenido\ASIGNA...,2024-09-24 11:50:36.078466,2024-09-24 11:50:38.667874,2025-06-12 15:36:25.238969,113,00:07:27,447120,h264,2000,25.0,1920x1080,aac,125,1,44100
210,020 Gestion Color Management - 100 Flujos de t...,\\animum.local\Produccion\LOL-Contenido\ASIGNA...,2024-09-24 11:42:14.313174,2024-09-24 11:42:16.176010,2025-06-12 15:36:25.347139,82,00:05:23,323000,h264,2003,25.0,1920x1080,aac,125,1,44100


In [ ]:
from dpm_utils_functions.dpm_tables import table_various_sources_to_DF
from dpm_common_functions import _ini_authenticate_API

# Configuración
params_dic = {
    "spreadsheet_source_table_id": "docs.google.com/spreadsheets/d/1SzrRm8nUPPZqU_LE-xdDNdB7rLUdePkjgV4IuPA2U3o",
    "spreadsheet_source_table_worksheet_name": "ARCHIVOS LIST",

    "ini_environment_identificated": ini_environment_identificated,
    "json_keyfile_local": GCP_json_keyfile_local,
    "json_keyfile_colab": GCP_json_keyfile_colab,
    "json_keyfile_GCP_secret_id": GCP_json_keyfile_GCP_secret_id,
}



df_original = table_various_sources_to_DF(params_dic)
df_original.info()

[EXTRACTION [START ⏳]] Extrayendo datos de Google Sheets...
[EXTRACTION [SUCCESS ✅]] Datos extraídos con éxito de la hoja 'ARCHIVOS LIST'.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6067 entries, 0 to 6066
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   file_name                6067 non-null   object
 1   file_path                6067 non-null   object
 2   file_creation_date       6067 non-null   object
 3   file_last_modified_date  6067 non-null   object
 4   file_scrap_date          6067 non-null   object
 5   file_size_mb             6067 non-null   object
 6   duration_hms             6067 non-null   object
 7   duration_ms              6067 non-null   object
 8   video_codec              6067 non-null   object
 9   video_bitrate_kbps       6067 non-null   object
 10  video_fps                6067 non-null   object
 11  video_resolution         6067 non-null   object
 12  audio_c

In [ ]:
config = {
    "reference_dtype_df": df_videos,   # DataFrame con los dtypes de referencia
    "target_dtype_df": df_original    # DataFrame al que se le aplicarán esos dtypes
}
from dpm_utils_functions.dpm_tables import DType_df_to_df
df_original_casted, meta = DType_df_to_df(config)


🔹🔹🔹 [START ▶️] DTYPE COPY
[DTYPE COPY ℹ️] (1/16) 'file_name' ya es object. Skipped.
[DTYPE COPY ℹ️] (2/16) 'file_path' ya es object. Skipped.
[DTYPE COPY ✅] (3/16) 'file_creation_date' → datetime64[ns].
[DTYPE COPY ✅] (4/16) 'file_last_modified_date' → datetime64[ns].
[DTYPE COPY ✅] (5/16) 'file_scrap_date' → datetime64[ns].
[DTYPE COPY ✅] (6/16) 'file_size_mb' → int64.
[DTYPE COPY ℹ️] (7/16) 'duration_hms' ya es object. Skipped.
[DTYPE COPY ✅] (8/16) 'duration_ms' → Int64.
[DTYPE COPY ℹ️] (9/16) 'video_codec' ya es object. Skipped.
[DTYPE COPY ✅] (10/16) 'video_bitrate_kbps' → int64.
[DTYPE COPY ✅] (11/16) 'video_fps' → float64.
[DTYPE COPY ℹ️] (12/16) 'video_resolution' ya es object. Skipped.
[DTYPE COPY ℹ️] (13/16) 'audio_codec' ya es object. Skipped.
[DTYPE COPY ✅] (14/16) 'audio_bitrate_kbps' → int64.
[DTYPE COPY ✅] (15/16) 'audio_channels' → int64.
[DTYPE COPY ✅] (16/16) 'audio_sample_rate_hz' → int64.
[DTYPE COPY ✔️] Cast fin — ok: 10/16 | fail: 0 | skipped: 6
<class 'pandas.cor

In [ ]:
# 1️⃣ Configuración

config = {
    "validate_df_schemas_match" : True, # valida esquema primero

    "df_initial"   : df_original_casted,
    "df_to_merge"  : df_videos,
    "id_fields"    : ["file_name", "duration_ms"],

    "duplicate_policy": "keep_newest",   # keep_newest | keep_oldest | keep_df_initial | keep_df_to_merge
    "duplicate_date_field"       : "file_scrap_date",
    "duplicate_date_field_format_str"  : "%d/%m/%Y",


    "return_metadata"  : True
}


# Llamada a la función
from dpm_utils_functions.dpm_tables import tables_consolidate_duplicates_df
result_df, metadata = tables_consolidate_duplicates_df(config)



[CONSOLIDATION START ▶️] 2025-06-12T15:46:46
INFO ℹ️ id_fields=['file_name', 'duration_ms'] | policy=keep_newest
FINISHED ✅ registros finales=6064 | duplicados_resueltos=215


In [ ]:
# @title SUBIDA A GOOGLE SHEETS
config = {
    "df": result_df,
    'spreadsheet_target_table_id': 'https://docs.google.com/spreadsheets/d/1SzrRm8nUPPZqU_LE-xdDNdB7rLUdePkjgV4IuPA2U3o',
    # Nombre de la pestaña (worksheet) destino
    'spreadsheet_target_table_worksheet_name': 'ResponseAPI',

    "ini_environment_identificated": ini_environment_identificated,
    "json_keyfile_local": GCP_json_keyfile_local,
    "json_keyfile_colab": GCP_json_keyfile_colab,
    "json_keyfile_GCP_secret_id": GCP_json_keyfile_GCP_secret_id,
}
from dpm_utils_functions.dpm_tables import table_DF_to_various_targets

table_DF_to_various_targets(config)



🔹🔹🔹 [START ▶️] Iniciando escritura de DataFrame en destino configurado 🔹🔹🔹

[METRICS [INFO ℹ️]] DataFrame recibido: 6064 filas × 16 columnas.

[LOAD [START ▶️]] Iniciando escritura en Google Sheets…
[LOAD [SUCCESS ✅]] Se actualizaron N/A celdas en Google Sheets.
[METRICS [INFO ℹ️]] Destino final: https://docs.google.com/spreadsheets/d/1SzrRm8nUPPZqU_LE-xdDNdB7rLUdePkjgV4IuPA2U3o

🔹🔹🔹 [END [FINISHED ✅]] Escritura completada exitosamente. 🔹🔹🔹

